**Note:** This is a DDL notebook. Run this only once

In [1]:
from pyspark.sql import SparkSession

import os

from seed.unity import conf

spark: SparkSession = SparkSession.builder.config(conf=conf).getOrCreate()
print(f"Spark {spark.version} is up and running!")

# spark.sql("USE unity;").show()

:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
io.delta#delta-spark_2.12 added as a dependency
org.apache.iceberg#iceberg-spark-runtime-3.5_2.12 added as a dependency
software.amazon.awssdk#bundle added as a dependency
org.slf4j#slf4j-simple added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
com.amazonaws#aws-java-sdk-bundle added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-07612ac9-a874-47fb-a298-96d084ee87bc;1.0
	confs: [default]
	found io.delta#delta-spark_2.12;3.3.1 in central
	found io.delta#delta-storage;3.3.1 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
	found org.apache.iceberg#iceberg-spark-runtime-3.5_2.12;1.5.0 in central
	found software.amazon.awssdk#bundle;2.29.52 in central
	found org.slf4j#slf4j-simple;2.0.7 in central
	found org.slf4j#slf4j-api;2.0.7 in central
	found org.apache.hadoop#hadoop-aws;3.3.4 in central
	found org.wildfly.openssl#wildfl

Spark 3.5.2 is up and running!


25/04/29 08:50:09 WARN SparkSession: Cannot use org.apache.iceberg.spark.extensions.IcebergSparkSessionExtension to configure session extensions.
java.lang.ClassNotFoundException: org.apache.iceberg.spark.extensions.IcebergSparkSessionExtension
	at java.base/java.net.URLClassLoader.findClass(URLClassLoader.java:476)
	at java.base/java.lang.ClassLoader.loadClass(ClassLoader.java:594)
	at java.base/java.lang.ClassLoader.loadClass(ClassLoader.java:527)
	at java.base/java.lang.Class.forName0(Native Method)
	at java.base/java.lang.Class.forName(Class.java:398)
	at org.apache.spark.util.SparkClassUtils.classForName(SparkClassUtils.scala:41)
	at org.apache.spark.util.SparkClassUtils.classForName$(SparkClassUtils.scala:36)
	at org.apache.spark.util.Utils$.classForName(Utils.scala:94)
	at org.apache.spark.sql.SparkSession$.$anonfun$applyExtensions$2(SparkSession.scala:1367)
	at org.apache.spark.sql.SparkSession$.$anonfun$applyExtensions$2$adapted(SparkSession.scala:1365)
	at scala.collection.mu

In [2]:
print("unity catalog config:")
for k, v in spark.sparkContext.getConf().getAll():
    if k.startswith("spark.sql.catalog"):
        print(f"  {k} = {v!r}")

spark.sql("SHOW CATALOGS").show()


unity catalog config:
  spark.sql.catalog = 'org.apache.iceberg.spark.SparkSessionCatalog'
  spark.sql.catalog.type = 'rest'
  spark.sql.catalog.uri = 'http://iceberg-rest:8181'
  spark.sql.catalog.s3.path-style-access = 'true'
  spark.sql.catalog.s3.endpoint = 'http://minio:9000'
  spark.sql.catalog.warehouse = 's3a://iceberg/'
  spark.sql.catalog.s3.secret-access-key = 'miniouser'
  spark.sql.catalog.catalog-impl = 'org.apache.iceberg.rest.RESTCatalog'
  spark.sql.catalog.s3.access-key-id = 'miniouser'
  spark.sql.catalog.s3.region = 'us-east-1'
+-------------+
|      catalog|
+-------------+
|spark_catalog|
+-------------+



In [3]:
# Create a namespace in unity catalog

# spark.sql("DROP NAMESPACE IF EXISTS dev;")

spark.sql("CREATE NAMESPACE IF NOT EXISTS dev;")

spark.sql("SHOW NAMESPACES;").show()

+---------+
|namespace|
+---------+
|  default|
|      dev|
+---------+



In [4]:
# Read flights data from parquet file

df = spark.read.parquet("s3a://seed/flights-1m.parquet")
df.show(5)

df.printSchema()

df.createOrReplaceTempView("raw_flights")

spark.sql("""
    SELECT
        MIN(FL_DATE) AS min_date,
        MAX(FL_DATE) AS max_date,
        COUNT(*) AS num_rows
    FROM raw_flights;
""").show()

spark.sql("""
    SELECT
        FL_DATE,
        COUNT(*) AS num_rows
    FROM raw_flights
    GROUP BY FL_DATE
    ORDER BY FL_DATE
    LIMIT 5;
""").show()

25/04/29 08:50:27 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


+----------+---------+---------+--------+--------+---------+---------+
|   FL_DATE|DEP_DELAY|ARR_DELAY|AIR_TIME|DISTANCE| DEP_TIME| ARR_TIME|
+----------+---------+---------+--------+--------+---------+---------+
|2006-01-01|        5|       19|     350|    2475| 9.083333|12.483334|
|2006-01-02|      167|      216|     343|    2475|11.783334|15.766666|
|2006-01-03|       -7|       -2|     344|    2475| 8.883333|12.133333|
|2006-01-04|       -5|      -13|     331|    2475| 8.916667|    11.95|
|2006-01-05|       -3|      -17|     321|    2475|     8.95|11.883333|
+----------+---------+---------+--------+--------+---------+---------+
only showing top 5 rows

root
 |-- FL_DATE: date (nullable = true)
 |-- DEP_DELAY: short (nullable = true)
 |-- ARR_DELAY: short (nullable = true)
 |-- AIR_TIME: short (nullable = true)
 |-- DISTANCE: short (nullable = true)
 |-- DEP_TIME: float (nullable = true)
 |-- ARR_TIME: float (nullable = true)

+----------+----------+--------+
|  min_date|  max_date|n

In [5]:
# Create flights table from parquet file

spark.sql("""
    CREATE TABLE IF NOT EXISTS dev.flights
    USING iceberg
    PARTITIONED BY (fl_date)
    AS
    SELECT
        *
    FROM raw_flights;
""")

spark.sql("SELECT * FROM dev.flights LIMIT 5;").show()

# spark.sql("DROP TABLE dev.flights PURGE;")

25/04/29 08:50:32 ERROR Utils: Aborting task
org.apache.spark.sql.AnalysisException: Table dev.flights does not support append in batch mode.;
AppendData RelationV2[FL_DATE#112, DEP_DELAY#113, ARR_DELAY#114, AIR_TIME#115, DISTANCE#116, DEP_TIME#117, ARR_TIME#118] spark_catalog.dev.flights dev.flights, false
+- Project [FL_DATE#21, DEP_DELAY#22, ARR_DELAY#23, AIR_TIME#24, DISTANCE#25, DEP_TIME#26, ARR_TIME#27]
   +- SubqueryAlias raw_flights
      +- View (`raw_flights`, [FL_DATE#21,DEP_DELAY#22,ARR_DELAY#23,AIR_TIME#24,DISTANCE#25,DEP_TIME#26,ARR_TIME#27])
         +- Relation [FL_DATE#21,DEP_DELAY#22,ARR_DELAY#23,AIR_TIME#24,DISTANCE#25,DEP_TIME#26,ARR_TIME#27] parquet

	at org.apache.spark.sql.errors.QueryCompilationErrors$.unsupportedTableOperationError(QueryCompilationErrors.scala:1277)
	at org.apache.spark.sql.errors.QueryCompilationErrors$.unsupportedAppendInBatchModeError(QueryCompilationErrors.scala:1289)
	at org.apache.spark.sql.execution.datasources.v2.TableCapabilityCheck$.$

AnalysisException: Table dev.flights does not support append in batch mode.;
AppendData RelationV2[FL_DATE#112, DEP_DELAY#113, ARR_DELAY#114, AIR_TIME#115, DISTANCE#116, DEP_TIME#117, ARR_TIME#118] spark_catalog.dev.flights dev.flights, false
+- Project [FL_DATE#21, DEP_DELAY#22, ARR_DELAY#23, AIR_TIME#24, DISTANCE#25, DEP_TIME#26, ARR_TIME#27]
   +- SubqueryAlias raw_flights
      +- View (`raw_flights`, [FL_DATE#21,DEP_DELAY#22,ARR_DELAY#23,AIR_TIME#24,DISTANCE#25,DEP_TIME#26,ARR_TIME#27])
         +- Relation [FL_DATE#21,DEP_DELAY#22,ARR_DELAY#23,AIR_TIME#24,DISTANCE#25,DEP_TIME#26,ARR_TIME#27] parquet
